In [1]:
import pandas as pd
import numpy as np
import country_converter as coco
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import nltk

%matplotlib inline

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Component: Raw Data Source

## Step 1. Select
From a variety of Data sources, we had to decide which raw data to use that can fit our interest and choose one data set to explore about the content. Finally we chose the salary data for 2023, we can study a more recent trend about the labor market which helps us to understand our strength.

In [2]:
df = pd.read_csv('ds_salaries.csv')

## Step 2. Identify
Identify what types of data that we are using and its applicability. In our data, we have numbers, integers, decimals and texts which are arranged by different columns to assist us to have a more clear view of the data that we can use to answer the questions we originally defined.

- **Identitfy what types of data we are using**
- **Give a more clear view by filling more complete information**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


In [4]:
df['experience_level'].value_counts()

experience_level
SE    2516
MI     805
EN     320
EX     114
Name: count, dtype: int64

In [5]:
employment_type = 'employment_type'
df[employment_type] = df[employment_type].replace('FT','Full-Time')
df[employment_type] = df[employment_type].replace('CT','Contract')
df[employment_type] = df[employment_type].replace('PT','Part-Time')
df[employment_type] = df[employment_type].replace('FL','Freelance')
df[employment_type].value_counts()

employment_type
Full-Time    3718
Part-Time      17
Contract       10
Freelance      10
Name: count, dtype: int64

In [6]:
experience_level = 'experience_level'
df[experience_level] = df[experience_level].replace('EN','Entry-level/Junior')
df[experience_level] = df[experience_level].replace('MI','Mid-level/Intermediate')
df[experience_level] = df[experience_level].replace('SE','Senior-level/Expert')
df[experience_level] = df[experience_level].replace('EX','Executive-level/Director')
df[experience_level].value_counts()

experience_level
Senior-level/Expert         2516
Mid-level/Intermediate       805
Entry-level/Junior           320
Executive-level/Director     114
Name: count, dtype: int64

## Step 3. Check 
Considering how to keep the data and its functions to use. We considered putting data in github to store our data in a structured way and easy to reach, to import and understand. 
- **We put the data on github for single source of truth and easy to access.**
- **Check the data of row is complete, and no null value.**

## Step 4. Access
Ensure the access of data and the sources suit the problem we defined at the first beginning. Here, when we were processing data, we were checking if the model can be run smoothly.
- **We can access the data smoothly and the data is in a good format to use.**

In [7]:
print(df.shape)
df.head()

(3755, 11)


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,Senior-level/Expert,Full-Time,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,Mid-level/Intermediate,Contract,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,Mid-level/Intermediate,Contract,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,Senior-level/Expert,Full-Time,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,Senior-level/Expert,Full-Time,Data Scientist,120000,USD,120000,CA,100,CA,M


In [8]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

## Step 5. Extract
Pick up the relevant data to use and ensure the data can be accessed without problem.
* **We select the necessary columns to use and drop the columns that are not useful for our analysis.**

In [9]:
df.drop(df[['salary','salary_currency']], axis = 1, inplace = True)
df

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,Senior-level/Expert,Full-Time,Principal Data Scientist,85847,ES,100,ES,L
1,2023,Mid-level/Intermediate,Contract,ML Engineer,30000,US,100,US,S
2,2023,Mid-level/Intermediate,Contract,ML Engineer,25500,US,100,US,S
3,2023,Senior-level/Expert,Full-Time,Data Scientist,175000,CA,100,CA,M
4,2023,Senior-level/Expert,Full-Time,Data Scientist,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...
3750,2020,Senior-level/Expert,Full-Time,Data Scientist,412000,US,100,US,L
3751,2021,Mid-level/Intermediate,Full-Time,Principal Data Scientist,151000,US,100,US,L
3752,2020,Entry-level/Junior,Full-Time,Data Scientist,105000,US,100,US,S
3753,2020,Entry-level/Junior,Contract,Business Data Analyst,100000,US,100,US,L


# Use Case 1 - Average Salary based on Company Location

## Component: Explorative Data Analysis

### Step 6. Select
After extracting the most important data, we have already selected those that can be analyzed and to dive into different characteristics about the data, or our employment data to build preparation strategies or to make sure the validity of the result.
- **We selected the use case that we are interested in for use case 1.**
- **We output the necessary data, and saving to the github.**

In [10]:
selected_columns = ['employee_residence', 'company_location', 'salary_in_usd']
uc1 = df[selected_columns]


# output data to csv for use case 1 
uc1.to_csv('uc1.csv', index=False)
uc1

,employee_residence,company_location,salary_in_usd
0,ES,ES,85847
1,US,US,30000
2,US,US,25500
3,CA,CA,175000
4,CA,CA,120000
...,...,...,...
3750,US,US,412000
3751,US,US,151000
3752,US,US,105000
3753,US,US,100000


### Step 7. Identity
To identify which kind of data that we can use also helps ourselves to prepare the data to be analyzed. After we choose the columns and data characteristics which fit to use for our problem to be solved or questions to be answered, we can prepare the analysis better than directly diving into it.

- **Access the data from Data Management, which is from github.**
- **We identify the country code by country_converter libaray.**
- **Printout the average salary by company location.**


In [11]:
uc1 = pd.read_csv('uc1.csv')

In [12]:
uc1['company_location'].value_counts()

company_location
US    3040
GB     172
CA      87
ES      77
IN      58
      ... 
MK       1
BS       1
IR       1
CR       1
MT       1
Name: count, Length: 72, dtype: int64

In [13]:
# Calculating the average salary for each city based on 'employee_residence'
average_salaries_desc = uc1.groupby('company_location')['salary_in_usd'].mean().sort_values(ascending=False).reset_index()
average_salaries_desc


,company_location,salary_in_usd
0,IL,271446.500000
1,PR,167500.000000
2,US,151822.009539
3,RU,140333.333333
4,CA,131917.689655
...,...,...
67,VN,12000.000000
68,AL,10000.000000
69,MA,10000.000000
70,BO,7500.000000


In [14]:
country = coco.convert(names = df['employee_residence'], to = "ISO3")
uc1['employee_residence'] = country
uc1['company_location'] = country
uc1['employee_residence'].value_counts()

employee_residence
USA    3004
GBR     167
CAN      85
ESP      80
IND      71
       ... 
BIH       1
ARM       1
CYP       1
KWT       1
MLT       1
Name: count, Length: 78, dtype: int64

## Data Preparation

### Step 8. Check 
When arranging the data, the requirement should be checked. 

**Requirement**
1. No null value. 
2. Data can be used. 

In [15]:
uc1.isnull().sum()

employee_residence    0
company_location      0
salary_in_usd         0
dtype: int64

In [16]:
uc1

,employee_residence,company_location,salary_in_usd
0,ESP,ESP,85847
1,USA,USA,30000
2,USA,USA,25500
3,CAN,CAN,175000
4,CAN,CAN,120000
...,...,...,...
3750,USA,USA,412000
3751,USA,USA,151000
3752,USA,USA,105000
3753,USA,USA,100000


# Use Case 7 - Relationship Between Employee Location and Company

### Step 6. Select

In [17]:
selected_columns = ['employee_residence', 'company_location']
uc7 = df[selected_columns]
# output data to csv for use case 7
uc7.to_csv('uc7.csv', index=False)
uc7

,employee_residence,company_location
0,ES,ES
1,US,US
2,US,US
3,CA,CA
4,CA,CA
...,...,...
3750,US,US
3751,US,US
3752,US,US
3753,US,US


### Step 7. Identity

In [18]:
country = coco.convert(names = uc7['employee_residence'], to = "ISO3")
uc7['employee_residence'] = country
uc7

,employee_residence,company_location
0,ESP,ES
1,USA,US
2,USA,US
3,CAN,CA
4,CAN,CA
...,...,...
3750,USA,US
3751,USA,US
3752,USA,US
3753,USA,US


In [21]:
residence = uc7['employee_residence'].value_counts()
fig = px.choropleth(locations = residence.index,
                    color = residence.values,
                    color_continuous_scale=px.colors.sequential.YlGn,
                    title = 'Employee Loaction On Map')
fig.show()


### Most of the employees are from USA as can be seen from the chloropleth above.

In [22]:
top_15_emp_locations = residence[:15]
fig = px.bar(y = top_15_emp_locations.values, x = top_15_emp_locations.index, 
            color = top_15_emp_locations.index, text = top_15_emp_locations.values,
            title = 'Top 15 Locations of Employees')

fig.update_layout( xaxis_title = "Location of Employees", yaxis_title = "count")
fig.show()

#### In terms of number of employees, USA leads followed by GBR, ESP and then IND.

In [23]:
country = coco.convert(names=df['company_location'], to="ISO3")
df['company_location'] = country
company_location = df['company_location'].value_counts()
top_15_company_location = company_location[:15]

fig = go.Figure(data = [
    go.Bar(name = 'Employee Residence', 
           x = top_15_emp_locations.index, y = top_15_emp_locations.values,
           text = top_15_emp_locations.values),
    go.Bar(name = 'Company Location', x = top_15_company_location.index, 
           y = top_15_company_location.values, text = top_15_company_location.values)])

fig.update_layout(barmode = 'group', xaxis_tickangle = -45,
                  title='Comparison of Employee Residence and Company Location')

fig.show()

#### As of 2023, most countries have similar number of employee residences and company locations.

### Step 8. Check

In [24]:
uc7.isnull().sum()

employee_residence    0
company_location      0
dtype: int64

In [25]:
uc7

,employee_residence,company_location
0,ESP,ES
1,USA,US
2,USA,US
3,CAN,CA
4,CAN,CA
...,...,...
3750,USA,US
3751,USA,US
3752,USA,US
3753,USA,US


# Use Case 8 - Company Size Ratios

### Step 6. Select

In [26]:
selected_columns = ['company_size']
uc8 = df[selected_columns]
# output data to csv for use case 8
uc8.to_csv('uc8.csv', index=False)
uc8

,company_size
0,L
1,S
2,S
3,M
4,M
...,...
3750,L
3751,L
3752,S
3753,L


### Step 7. Identity

In [27]:
group = uc8['company_size'].value_counts()

fig = px.bar(y = group.values, x = group.index, 
             color = group.index, text = group.values,
             title = 'Distribution of Company Size')

fig.update_layout(xaxis_title = "Company Size", yaxis_title = "count")
fig.show()

##### Company size mostly consists of medium sized comapnies and then the large sized ones followed by the small sized startups.

### Step 8.Check

In [28]:
uc8.isnull().sum()

company_size    0
dtype: int64

In [29]:
uc8

,company_size
0,L
1,S
2,S
3,M
4,M
...,...
3750,L
3751,L
3752,S
3753,L


# Use Case 9 - Development in the Field of Data Science

### Step 6. Select

### Step 7. Identity

### Step 8.Check

# Use Case 10 - Salary Range Levels

### Step 6. Select

In [31]:
selected_columns = ['salary_in_usd']
uc10 = df[selected_columns]
# output data to csv for use case 10
uc10.to_csv('uc10.csv', index=False)
uc10

,salary_in_usd
0,85847
1,30000
2,25500
3,175000
4,120000
...,...
3750,412000
3751,151000
3752,105000
3753,100000


### Step 7.  Identity

#### Box plot: Show the distribution of salary, it can display the median, quartiles, minimum, maximum and outliers. 
The median is $135,000.
The interquartile range shows that most of the data is concentrated between $95,000 and $175,000
The lower fence is equal to the minimum value, which may indicate that there is a fixed threshold for the salary floor, such as the industry standard minimum wage

In [32]:
fig = px.box(y = uc10['salary_in_usd'], title = 'Salary in USD')
fig.show()

In [38]:
hist_data = [uc10['salary_in_usd']]
group_labels = ['salary_in_usd']

fig = ff.create_distplot(hist_data, group_labels, show_hist = False)
fig.update_layout(title = 'Distribution Plot of Salary in USD') 
fig.show()

##### We observe that salary in USD is mostly distributed between 95/100k - 175/180k.

### Step 8.Check

In [35]:
uc10.isnull().sum()

salary_in_usd    0
dtype: int64

In [42]:
uc10

,salary_in_usd
0,85847
1,30000
2,25500
3,175000
4,120000
...,...
3750,412000
3751,151000
3752,105000
3753,100000


# Use Case 11 - Work Modes (Fully Remote / Remote / No Remote)

### Step 6. Select

In [40]:
selected_columns = ['remote_ratio']
uc11 = df[selected_columns]
# output data to csv for use case 11
uc11.to_csv('uc11.csv', index=False)
uc11

,remote_ratio
0,100
1,100
2,100
3,100
4,100
...,...
3750,100
3751,100
3752,100
3753,100


### Step 7. Identity

##### Remote Ratio consists of 3 values : 
##### 0 : No remote work (less than 20%)
##### 50 : Partially remote
##### 100 : Fully remote (more than 80%)

In [41]:
remote_type = ['Fully Remote', 'Partially Remote', 'No Remote Work']

fig = px.bar(x = remote_type, y = df['remote_ratio'].value_counts().values,
       color = remote_type, text = df['remote_ratio'].value_counts().values,
       title = 'Remote Ratio Distribution')

fig.update_layout( xaxis_title = "Remote Type", yaxis_title = "count")
fig.show()

##### As of 2023, Most companies still go the Fully Remote route, followed closely by the Hybrid/ Partially remote and then the No Remote policy.

### Step 8.Check

In [43]:
uc11.isnull().sum()

remote_ratio    0
dtype: int64

In [44]:
uc11

,remote_ratio
0,100
1,100
2,100
3,100
4,100
...,...
3750,100
3751,100
3752,100
3753,100


# Use Case 12 - Relationship Between Work Mode and Work Experience

### Step 6. Select

In [45]:
selected_columns = ['remote_ratio', 'work_year']
uc12 = df[selected_columns]
# output data to csv for use case 12
uc12.to_csv('uc12.csv', index=False)
uc12

,remote_ratio,work_year
0,100,2023
1,100,2023
2,100,2023
3,100,2023
4,100,2023
...,...,...
3750,100,2020
3751,100,2021
3752,100,2020
3753,100,2020


### Step 7.  Identity

##### We do this to check whether remote ratio gets affected over time. -Remote Ratio by Work Year

In [54]:
remote_year = uc12.groupby(['work_year','remote_ratio']).size()
ratio_2020 = np.round(remote_year[2020].values/remote_year[2020].values.sum(),2)
ratio_2021 = np.round(remote_year[2021].values/remote_year[2021].values.sum(),2)
ratio_2022 = np.round(remote_year[2022].values/remote_year[2022].values.sum(),2)
ratio_2023 = np.round(remote_year[2023].values/remote_year[2023].values.sum(),2)

fig = go.Figure()
categories = ['No Remote Work', 'Partially Remote', 'Fully Remote']
fig.add_trace(go.Scatterpolar(
            r = ratio_2020, theta = categories, 
            fill = 'toself', name = '2020 remote ratio'))

fig.add_trace(go.Scatterpolar(
            r = ratio_2021, theta = categories,
            fill = 'toself', name = '2021 remote ratio'))

fig.add_trace(go.Scatterpolar(
            r = ratio_2022, theta = categories,
            fill = 'toself', name = '2022 remote ratio'))

fig.add_trace(go.Scatterpolar(
            r = ratio_2023, theta = categories,
            fill = 'toself', name = '2023 remote ratio'))

fig.show()

##### As from the rader plot, we observe that :
##### Remote work peaked during 2021 and that could be attributed to the pandemic. With time, no remote work policy has been applied again as it is the case for 2023.

### Step 8. Check

In [55]:
uc12.isnull().sum()

remote_ratio    0
work_year       0
dtype: int64

In [56]:
uc12

,remote_ratio,work_year
0,100,2023
1,100,2023
2,100,2023
3,100,2023
4,100,2023
...,...,...
3750,100,2020
3751,100,2021
3752,100,2020
3753,100,2020


# Use Case 13 - Relationship Between Company Size and Work Experience

### Step 6.Select

In [57]:
selected_columns = ['experience_level','company_size']
uc13 = df[selected_columns]
# output data to csv for use case 13
uc13.to_csv('uc13.csv', index=False)
uc13

,experience_level,company_size
0,Senior-level/Expert,L
1,Mid-level/Intermediate,S
2,Mid-level/Intermediate,S
3,Senior-level/Expert,M
4,Senior-level/Expert,M
...,...,...
3750,Senior-level/Expert,L
3751,Mid-level/Intermediate,L
3752,Entry-level/Junior,S
3753,Entry-level/Junior,L


### Step 7.  Identity

In [58]:
exp_size = uc13.groupby(['experience_level','company_size']).size()
fig = go.Figure(data = [
    go.Bar(name = 'Entry-level/Junior', x = exp_size['Entry-level/Junior'].index,
           y = exp_size['Entry-level/Junior'].values, text = exp_size['Entry-level/Junior'].values),
    go.Bar(name = 'Executive-level/Director', x = exp_size['Executive-level/Director'].index,
           y = exp_size['Executive-level/Director'].values, text = exp_size['Executive-level/Director'].values),
    go.Bar(name = 'Mid-level/Intermediate', x = exp_size['Mid-level/Intermediate'].index,
           y = exp_size['Mid-level/Intermediate'].values, text = exp_size['Mid-level/Intermediate'].values),
    go.Bar(name = 'Senior-level/Expert', x = exp_size['Senior-level/Expert'].index,
           y = exp_size['Senior-level/Expert'].values, text = exp_size['Senior-level/Expert'].values),
])
fig.update_layout(xaxis_tickangle = -45, title = 'Experience Level with Company Size')

fig.show()

##### For large sized companies, there's mostly seniors followed by the intermediate and then the freshers. Very few executives can be seen.
##### For medium sized companies, it primarily comprises of senior level then followed by the intermediate and then the entry levelers. Executive a bit prominent compared to that of the large sized companies.
##### For small sized companies / startups, we see it consists all three levels apart from executives proportionally.

### Step 8. Check

In [59]:
uc12.isnull().sum()

remote_ratio    0
work_year       0
dtype: int64

In [60]:
uc12

,remote_ratio,work_year
0,100,2023
1,100,2023
2,100,2023
3,100,2023
4,100,2023
...,...,...
3750,100,2020
3751,100,2021
3752,100,2020
3753,100,2020


# Component: Data Management

### Step 9. Structure
To check the infrastructure for our project to be run without problem, we assess our devices for the analysis before start. Either the computer, or other internet that we are using.
- **We check the infrastructure for our project to be run without problem.**
- **Make sure we can access the data from data management.**

In [ ]:
import os
import psutil  # for checking memory usage

print("CPU usage:", psutil.cpu_percent(interval=1), "%")  # every 1 second cpu usage
print("memory usage:", psutil.virtual_memory())  # memory usage


CPU usage: 32.1 %
memory usage: svmem(total=17179869184, available=1239564288, percent=92.8, used=1801277440, free=13504512, active=1230143488, inactive=1217159168, wired=571133952)


In [ ]:
import sys

print("Python version:", sys.version)
# check pandas version
import pandas as pd
print("Pandas version:", pd.__version__)


Python version: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:43:38) [Clang 15.0.7 ]
Pandas version: 2.1.3


In [ ]:
## access data from csv file
pd.read_csv('../data/uc1.csv')

,employee_residence,company_location,salary_in_usd
0,ESP,ESP,85847
1,USA,USA,30000
2,USA,USA,25500
3,CAN,CAN,175000
4,CAN,CAN,120000
...,...,...,...
3750,USA,USA,412000
3751,USA,USA,151000
3752,USA,USA,105000
3753,USA,USA,100000
